In [1]:
def makeFreqList (sceneWords):
    freqList = [0]
    wordToFreqDict = dict()
    for word in sceneWords:
        freq = 1
        if word in wordToFreqDict:
            oldFreq = wordToFreqDict[word]
            freqList[oldFreq] -=1
            freq = oldFreq+1
            
        wordToFreqDict[word] = freq
            
        if freq == len(freqList):
            freqList.append(1)
        else:
            freqList[freq]+=1
        
    return freqList

In [2]:
from __future__ import division

def makeYulesK (sceneWords):
    N = len (sceneWords)
    freqList = makeFreqList (sceneWords)
    summation = 0
    
    for m in range (1, len(freqList)):
        localProd = freqList[m] * m/N * m/N
        summation += localProd

    yulesK = -1/N + summation
    return yulesK
            

#Schema
Table name = yuleK
CREATE TABLE YULEK( ID VARCHAR(10) PRIMARY KEY NOT NULL, PLAY VARCHAR(5) NOT NULL, ACT INT NOT NULL, SCENE INT NOT NULL, PLAY_TYPE VARCHAR(20) NOT NULL, yulek DOUBLE PRECISION NOT NULL);

In [14]:
import psycopg2

def connectToDB():
    try:
        conn = psycopg2.connect("dbname='shakespeare' user='evelynting' host='localhost' password='password'")
        return conn
    except:
        print("couldn't connect to db")

In [15]:
comedies = ["Ado", "AYL", "Err", "MND", "Wiv"]
def putInDB( conn, play_name, scene, yulesK ):
    cursor = conn.cursor()
    act = scene[3]
    info_end = len(scene) - 4
    scene = scene[10 : info_end]
    id = play_name+act+scene
    play_type = "comedy" if play_name in comedies else "tragedy"
    cursor.execute("INSERT INTO yulek (id, play, act, scene, play_type, yulek) VALUES (%s, %s, %s, %s, %s, %s)", (id, play_name, act, scene, play_type, yulesK))
    conn.commit()

In [16]:
import os
import re
def processPlays(dir_name, plays):
    conn = connectToDB();
    for play_name in plays:
        play_dir = dir_name + '/' + play_name
        scenes = [f for f in os.listdir(play_dir)]
        for scene in scenes:
            with open(play_dir + '/' + scene) as file:
                text = file.read().lower()
                file.close()
                sceneWords = text.split()
                K = makeYulesK( sceneWords )
                putInDB(conn, play_name, scene, K)

In [18]:
dir_name = "/Users/evelynting/Desktop/thesis/cleanPlays"
cleanPlays = [f for f in os.listdir(dir_name) ]
processPlays(dir_name, cleanPlays)